<img src="https://github.com/Shubhwithai/GRE_Geometry_quiz/blob/main/Group%2042.png?raw=true" width="" height="50">

Educhain is a powerful Python package that leverages Generative AI to create engaging and personalized educational content. From generating multiple-choice questions to crafting comprehensive lesson plans, Educhain makes it easy to apply AI in various educational scenarios.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1JEcwCGfLM_4i3JJKHzMbYTP7KYUrRFyL?authuser=3#scrollTo=Je6BARehmNMK)




In [1]:
!pip install educhain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of opentelemetry-proto to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of opentelemetry-proto to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-status to determi

In [1]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get("GEMINI_KEY")

In [2]:
from educhain import Educhain, LLMConfig
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
import os

gemini_flash = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash",
        google_api_key=GOOGLE_API_KEY
)
flash_config = LLMConfig(custom_model=gemini_flash)
client = Educhain(flash_config)

In [3]:
from PyPDF2 import PdfReader

def extract_text_from_pdf(file_path):
    reader = PdfReader(file_path)
    text = " ".join([page.extract_text() or "" for page in reader.pages])
    return " ".join(text.split())

def generate_questions_from_text(text, num_q=10, doubt=None):
    prompt = doubt if doubt else "Generate diverse questions from this PYQ"
    result = client.qna_engine.generate_questions_from_data(
        source=text,
        source_type="text",
        num=num_q,
        custom_instructions=(
            "Generate a mix of MCQs, True/False, Short and Long Answer questions based on this content. "
            "Add Bloom's taxonomy & difficulty levels where relevant."
        )
    )
    return result

def generate_mock_questions(exam_type, subject, topic="", num_q=10):
    """ Generate Mock - Practice question topic-wise using Educhain and Gemini """
    topic_query = f"{exam_type} {subject} {topic}".strip()
    result = client.qna_engine.generate_questions(
        topic=topic_query,
        num=num_q,
        custom_instructions=(
            "Generate diverse PYQ-style MCQ, TF, Short and Long answer questions with explanations, "
            "Bloom's levels, and difficulty rating."
        )
    )
    return result

def solve_image_doubt(image_path, prompt="Explain this image in detail"):
    explanation = client.qna_engine.solve_doubt(
        image_source=image_path,
        prompt=prompt,
        detail_level="High"
    )
    return explanation

pdf_path = "/content/IDB-PYQs.pdf"
text = extract_text_from_pdf(pdf_path)

print("Generating questions from uploaded PDF...")
questions = generate_questions_from_text(text, num_q=5)

for i, q in enumerate(questions.questions, 1):
    print(f"\nQ{i}: {q.question}")
    if hasattr(q, "options") and q.options:
        for j, opt in enumerate(q.options):
            print(f"  {chr(65 + j)}. {opt}")
        print(f"Answer: {q.answer}")
    else:
        print(f"Answer: {q.answer}")

    if getattr(q, "explanation", None):
        print(f"Explanation: {q.explanation}")


Generating questions from uploaded PDF...

Q1: Which of the following is NOT a characteristic of a well-defined project?
  A. Clear goals and objectives
  B. Specific scope and boundaries
  C. Measurable deliverables and outcomes
  D. Ambiguous goals and deliverables
Answer: Ambiguous goals and deliverables
Explanation: A well-defined project has clear goals, a specific scope, and measurable deliverables. Ambiguity is detrimental to project success.

Q2: What is the primary purpose of a Work Breakdown Structure (WBS)?
  A. To assign resources to project activities
  B. To identify project risks
  C. To create a project schedule
  D. To decompose project deliverables into smaller, more manageable tasks
Answer: To decompose project deliverables into smaller, more manageable tasks
Explanation: The WBS is a hierarchical decomposition of the project's deliverables, breaking down large tasks into smaller, more manageable components. This makes it easier to plan, estimate, and control the pro